In [43]:
import pandas as pd
from sklearn import preprocessing

#dataframe that keeps the data
df = pd.read_csv("wine.csv")
df.reset_index(inplace = True)
#set this to the desired number of inputs
numCentroids = 3

#set this to max iterations
maxIteration = 1000

#add cluster column
df['cluster'] = 0
data = df.values

In [46]:
'''
Get k centroids
Check each if san pinakamalapit
then average. then thats the new centroid
loop until converges or reached limit
'''
def cluster(numCentroids, maxIteration):
    centroids = data[:numCentroids]
    centMembCount[numCentroids] = 0 #centroid member count. used for averaging
    centFeatureSum[numCentroids][len(centoids[0]) - 1] = 0 #-1 because exclude cluster column
    
    #cluster the data. the cluster number is put on the last index of the row
    for i, row in enumerate(data):  
        centroid = getCentroid(centroids)
        data[i][len(row)-1] = centroid
        centMembCount[centroid] += 1 #adds cluster member count
        for j in range(len(row) - 1): #-1 because exclude cluster column. this loop adds the values to centFeatureSum
              centFeatureSum[centroid][j] += data[i][j]
    
    #gets new centroid values
    for i in range(numCentroids):
        for j in range(len(centoids[0]) - 1):
            centroids = centFeatureSum[i][j] / centMembCount[i]
            
            
    
        
    
    
    print(data)
    
'''
DO THISSSS
'''
def getCentroid(centroids):
    return 3333 
    
    
cluster(numCentroids, maxIteration)

[[  1.00000000e+00   1.42300000e+01   1.71000000e+00 ...,   3.92000000e+00
    1.06500000e+03   3.33300000e+03]
 [  1.00000000e+00   1.32000000e+01   1.78000000e+00 ...,   3.40000000e+00
    1.05000000e+03   3.33300000e+03]
 [  1.00000000e+00   1.31600000e+01   2.36000000e+00 ...,   3.17000000e+00
    1.18500000e+03   3.33300000e+03]
 ..., 
 [  3.00000000e+00   1.32700000e+01   4.28000000e+00 ...,   1.56000000e+00
    8.35000000e+02   3.33300000e+03]
 [  3.00000000e+00   1.31700000e+01   2.59000000e+00 ...,   1.62000000e+00
    8.40000000e+02   3.33300000e+03]
 [  3.00000000e+00   1.41300000e+01   4.10000000e+00 ...,   1.60000000e+00
    5.60000000e+02   3.33300000e+03]]


In [7]:
for index, row in df.iterrows():
    

index                     1.00
Alcohol                  14.23
MalicAcid                 1.71
Ash                       2.43
AlcalinityAsh            15.60
Magnesium               127.00
PhenolTotal               2.80
Flavanoids                3.06
NonflavanoidPhenols       0.28
Proanthocyanins           2.29
ColorIntensity            5.64
Hue                       1.04
D280                      3.92
Proline                1065.00
cluster                   0.00
Name: 0, dtype: float64
index                     1.00
Alcohol                  13.20
MalicAcid                 1.78
Ash                       2.14
AlcalinityAsh            11.20
Magnesium               100.00
PhenolTotal               2.65
Flavanoids                2.76
NonflavanoidPhenols       0.26
Proanthocyanins           1.28
ColorIntensity            4.38
Hue                       1.05
D280                      3.40
Proline                1050.00
cluster                   0.00
Name: 1, dtype: float64
index                 

In [10]:
data

array([[  1.00000000e+00,   1.42300000e+01,   1.71000000e+00, ...,
          3.92000000e+00,   1.06500000e+03,   0.00000000e+00],
       [  1.00000000e+00,   1.32000000e+01,   1.78000000e+00, ...,
          3.40000000e+00,   1.05000000e+03,   0.00000000e+00],
       [  1.00000000e+00,   1.31600000e+01,   2.36000000e+00, ...,
          3.17000000e+00,   1.18500000e+03,   0.00000000e+00],
       ..., 
       [  3.00000000e+00,   1.32700000e+01,   4.28000000e+00, ...,
          1.56000000e+00,   8.35000000e+02,   0.00000000e+00],
       [  3.00000000e+00,   1.31700000e+01,   2.59000000e+00, ...,
          1.62000000e+00,   8.40000000e+02,   0.00000000e+00],
       [  3.00000000e+00,   1.41300000e+01,   4.10000000e+00, ...,
          1.60000000e+00,   5.60000000e+02,   0.00000000e+00]])